In [22]:
!pip install datasets
!pip install --force-reinstall --no-deps git+https://github.com/huggingface/transformers
!pip install scikit-multilearn

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-tuovj9x5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-tuovj9x5
  Resolved https://github.com/huggingface/transformers to commit 24c91f095fec4d90fa6901ef17146b4f4c21d0a3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10328608 sha256=a09616f7b571d969a23ceb31a415a8e89e0eec974db7e5e39a1c58671838ae25
  Stored in directory: /tmp/pip-ephem-wheel-cache-3t8m22f_/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.0.dev0
    Uninstalling transformers-4.48.0.dev0:
      Successfully uninstalled transformers-4.48.0.dev0


In [23]:
import pandas as pd
import numpy as np
import nltk
import torch
import datasets
from datasets import Dataset, DatasetDict
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from typing import Union
from skmultilearn.model_selection import iterative_train_test_split

MODEL_NAME = "answerdotai/ModernBERT-base"

In [24]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [142]:
file = '/content/ptbr.csv'
data = pd.read_csv(file)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226 entries, 0 to 2225
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2226 non-null   object
 1   text      2226 non-null   object
 2   Anger     2226 non-null   int64 
 3   Disgust   2226 non-null   int64 
 4   Fear      2226 non-null   int64 
 5   Joy       2226 non-null   int64 
 6   Sadness   2226 non-null   int64 
 7   Surprise  2226 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 139.2+ KB


In [143]:
data.head()

,id,text,Anger,Disgust,Fear,Joy,Sadness,Surprise
0,ptbr_train_track_a_00001,"minha vó me disse que era frango e eu comi, ti...",0,0,0,0,1,0
1,ptbr_train_track_a_00002,Está e a nossa deputada Benedita linda guerrei...,0,0,0,1,0,0
2,ptbr_train_track_a_00003,só falta as roupas kkkkkkkkkkk,0,0,0,1,0,0
3,ptbr_train_track_a_00004,Eu tmb. Comecei a sair de casa agora (fui pela...,0,0,0,0,1,0
4,ptbr_train_track_a_00005,Peço a Deus que nossos dirigentes tenham realm...,0,0,0,0,0,0


### Pré-processamento dos dados
1. Case folding
2. Remover stop words
3. Remover acentuação e pontuação


In [144]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords[:20]

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele']

In [145]:
data["clean_text"] = data["text"].copy()
# Case folding
data["clean_text"] = data["clean_text"].str.lower()
# Remover stopwords
data["clean_text"] = data["clean_text"].replace({r"\b" + stopword + r"\b": "" for stopword in stopwords}, regex=True)
# Remover acentuação e pontuação
data["clean_text"] = data["clean_text"].str.replace(r"[\.!?\\\-.,]", "", regex=True)
data["clean_text"] = data["clean_text"].str.replace(r"\s+", " ", regex=True)


In [146]:
data["text"], data["clean_text"]

(0       minha vó me disse que era frango e eu comi, ti...
 1       Está e a nossa deputada Benedita linda guerrei...
 2                          só falta as roupas kkkkkkkkkkk
 3       Eu tmb. Comecei a sair de casa agora (fui pela...
 4       Peço a Deus que nossos dirigentes tenham realm...
                               ...                        
 2221              Eu acho que o CAP vai surpreender hein.
 2222    23:59 - Lula sabia de toda a corrupção no seu ...
 2223    O Brasil precisa URGENTE de pessoas sérias e c...
 2224    Sera que só eu acho que ta passando da hora de...
 2225                                     falta só 2 porra
 Name: text, Length: 2226, dtype: object,
 0         vó disse frango comi gosto frango mto inocente 
 1        deputada benedita linda guerreira parabéns ju...
 2                                falta roupas kkkkkkkkkkk
 3        tmb comecei sair casa agora ( primeira vez ci...
 4       peço deus dirigentes realmente iluminação toma...
              

### Adicionar emoção neutra

Para casos que não tem nenhuma emoção na linha, adicionamos uma nova emoção: neutro

In [147]:
BASE_EMOTIONS = ["Anger", "Disgust", "Fear", "Joy", "Sadness", "Surprise"]

data["Neutral"] = 0
no_emotions_mask = data[BASE_EMOTIONS].sum(axis=1) == 0
data.loc[no_emotions_mask, "Neutral"] = 1

In [148]:
EMOTIONS = BASE_EMOTIONS + ["Neutral"]
MAX_SENTENCE_LENGTH =data["clean_text"].str.len().max()

### Iterative train test split
Uma vez que o dataset é desbalanceado, precisamos garantir que os dados de treino e teste tenham proporções similares de cada classe. Entretanto, já que o nosso problema é multi classe, utilizar o *train_test_split* do scikit-learn não funciona, uma vez que ele não lida bem com problemas multi classe, pois nesse tipo de problema há muitas combinações de classe possíveis. Sendo assim, utilizamos a função *iterative_train_test_split* que se propõe a resolver esse problema

In [149]:
def concat_X_y(X: np.array, y: np.array, columns: list[str]) -> pd.DataFrame:
    concatted_np = np.concatenate((X, y), axis=1)
    concatted = pd.DataFrame(concatted_np, columns=columns)
    return concatted


def train_test_val_split(
        data: pd.DataFrame,
        feature_label: str,
        targets_labels: list[str]
    ) -> Union[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    X = data[feature_label].to_numpy().reshape(-1, 1)
    y = data[targets_labels].to_numpy()

    X_train, y_train, X_test, y_test = iterative_train_test_split(
        X,
        y,
        test_size=0.1,
    )
    X_train, y_train, X_val, y_val = iterative_train_test_split(
        X_train,
        y_train,
        test_size=0.1,
    )
    columns = [feature_label] + targets_labels

    train = concat_X_y(X_train, y_train, columns)
    test = concat_X_y(X_test, y_test, columns)
    val = concat_X_y(X_val, y_val, columns)

    return train, test, val

train, test, val = train_test_val_split(data, "clean_text", EMOTIONS)


### Preparação para a estrutura do Huggingface

In [150]:
def create_dataset_dict(train: pd.DataFrame, val: pd.DataFrame, test: pd.DataFrame) -> DatasetDict:
    train_dataset = Dataset.from_pandas(train)
    val_dataset = Dataset.from_pandas(val)
    test_dataset = Dataset.from_pandas(test)

    return DatasetDict({
        'train': train_dataset,
        'validation': val_dataset,
        'test': test_dataset
    })

dataset = create_dataset_dict(train, val, test)
print(dataset)
dataset["train"][0]


DatasetDict({
    train: Dataset({
        features: ['clean_text', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Neutral'],
        num_rows: 1803
    })
    validation: Dataset({
        features: ['clean_text', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Neutral'],
        num_rows: 201
    })
    test: Dataset({
        features: ['clean_text', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Neutral'],
        num_rows: 222
    })
})


{'clean_text': ' deputada benedita linda guerreira parabéns juntos',
 'Anger': 0,
 'Disgust': 0,
 'Fear': 0,
 'Joy': 1,
 'Sadness': 0,
 'Surprise': 0,
 'Neutral': 0}

In [151]:
labels = dataset['train'].column_names
labels.remove('clean_text')
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
print(labels)
print(id2label)
print(label2id)

['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Neutral']
{0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Joy', 4: 'Sadness', 5: 'Surprise', 6: 'Neutral'}
{'Anger': 0, 'Disgust': 1, 'Fear': 2, 'Joy': 3, 'Sadness': 4, 'Surprise': 5, 'Neutral': 6}


In [152]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

Usando o dispositivo: cuda


In [164]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def tokenize_data(examples):
  # take a batch of texts
  text = examples["clean_text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding
tokenized_dataset = dataset.map(tokenize_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/1803 [00:00<?, ? examples/s]

Map:   0%|          | 0/201 [00:00<?, ? examples/s]

Map:   0%|          | 0/222 [00:00<?, ? examples/s]

In [165]:
example = tokenized_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [166]:
tokenizer.decode(example['input_ids'])

'[CLS] deputada benedita linda guerreira parabens juntos [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [167]:
 tokenized_dataset['train']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1803
})

In [168]:
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

### Finetuning do modelo

In [169]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                    problem_type="multi_label_classification",
                                    num_labels=len(labels),
                                    id2label=id2label,
                                    label2id=label2id)
model.to(device)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModernBertForSequenceClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      

In [170]:
batch_size = 8
metric_name = "f1"

args = TrainingArguments(
    "bert-multisentiment-portuguese",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    label_names=["labels"],
    #push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1573: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [171]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [172]:
tokenized_dataset["train"][0]['labels']

tensor([0., 0., 0., 1., 0., 0., 0.])

In [173]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],  # Subsample para treino rápido
    eval_dataset=tokenized_dataset["validation"],    # Subsample para validação
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-173-a129b5fe8c84>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [174]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.save_model("/content/model")